# Modelo base charge cell Anglo American

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
import pickle
import pmdarima as pm
from ipywidgets import Button, HBox, VBox
from ipywidgets import interactive
import ipywidgets
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# Definición de tags relevantes
tag_granulometria=['CF:CVB007_S2.PNT.FOX']

tag_dispatch=['disp_crusher_index', 'disp_sag_power_index', 'disp_ball_work_index','disp_ley_calcopirita', 
             'disp_ley_pirita']

tag_metrica=['CF:EXPERTO:EXP_MS_A.MEAS_7.FOX','CF:EXPERTO:EXP_MS_A.MEAS_1.FOX','CF:215.WIC1605A.MEAS.FOX',
              'CF:EXPERTO:EXP_MS_A.MEAS_5.FOX','CF:225.WIC8067.MEAS.FOX']

In [3]:
# read
tags=pd.read_excel("../data/meta data/tags relevantes.xlsx")
tags_cc=tags.tag.to_list()
dic={}
for i,j in zip(tags.tag,tags.description):
    dic[i]=j

In [4]:
campañas=pd.read_csv("../data/meta data/campaign_29September2022.csv")
campañas["Fechas"]=campañas["Fechas"]+' 00:00:00'
campañas.Fechas=pd.to_datetime(campañas.Fechas)

In [5]:
inicios_campañas=campañas.Fechas.to_list()

In [6]:
# Se lee csv
liners_age=pd.read_csv('../data/processed data/liners_age_30September2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')

In [7]:
# Se lee csv
df=pd.read_csv('../data/processed data/cleaned_28September2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
df.sort_index(inplace=True)
#df.rename(columns=dic,inplace=True)
df=df.join(liners_age).interpolate()

### Carga de modelos

In [8]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//models//pickles//clasificador mineralogico.pkl', 'rb'))
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dic = pickle.load(open('..//models//pickles//Curvas de celda de carga vs granulometria productivo.pkl', 'rb'))

In [9]:
# F80
# Carga F80

# Funciones Auxiliares para que predicts esten entre 20 y 90.
def transform(x, a=20, b=90):
    return np.log((x-a)/(b-x))
def inverse_transform(y, a=20, b=90):
    return (b-a)/(1+np.exp(-y))+a

# Modelo auto ARIMA

def forecast_granulometry2(ts1, time_forward=1, freq='15T', certeza=95):
    #dejamos a la frecuencia de datos pedida
    ts1=ts1.groupby(pd.Grouper(freq=freq)).apply(np.nanmean)
    #Acá no necesitamos el tiempo directamente.
    #acá le aplicamos la transformación. Seguimos trabajando con una serie de tiempo.
    tmp=ts1.apply(lambda x: transform(x))
    #aca vemos si hay algo seasonal
    ft = np.fft.fft(tmp.values) / len(tmp.values)  # Normalize amplitude and apply the FFT
    ft = ft[range(int(len(tmp.values)/2))]   # Exclude sampling frequency
    frecuencia = np.arange(int(len(tmp.values) / 2)) / len(tmp.values)
    seasonal = int(1/pd.Series(abs(ft), index=frecuencia).iloc[2:100].idxmax())
    
    if seasonal > 3: #En este caso es true, ahora hagamos el pd autoarima con seasonal true
        model1 = pm.auto_arima(tmp, 
                      start_p=1, start_q=1, max_p=20, max_q=20, m=min(20,seasonal),
                             start_P=0, seasonal=True, d=1, D=1, trace=False,
                             error_action='ignore',  
                             suppress_warnings=True,
                             stepwise=True)
    else:
        model1 = pm.auto_arima(tmp, 
                      start_p=1, start_q=1, max_p=20, max_q=20,
                             start_P=0, seasonal=False, trace=False,
                             error_action='ignore',  
                             suppress_warnings=True,
                             stepwise=True)
    
    forecasts, conf = model1.predict(n_periods=len(pd.date_range(start=ts1.index[-1],
                        end=ts1.index[-1]+timedelta(hours=time_forward), freq=freq)[0:])-1,
                                     return_conf_int=True, alpha=(100-certeza)/100)
    ts = pd.Series(np.insert(forecasts,0,tmp.iloc[-1], axis=0),
          index=pd.date_range(start=ts1.index[-1],end=ts1.index[-1]+timedelta(hours=time_forward),
                              freq=freq)[0:]).apply(lambda x: inverse_transform(x))
    
    error=pd.DataFrame({'Lower_Bound':np.insert(conf[:,0], 0, tmp[ts1.index[-1]], axis=0),
    'Upper_Bound':np.insert(conf[:,1], 0,tmp[ts1.index[-1]], axis=0)},
        index=pd.date_range(start=ts1.index[-1],end=ts1.index[-1]+timedelta(hours=time_forward),
                            freq=freq)[0:]).apply(lambda x: inverse_transform(x) , axis=1)
    return pd.concat([ts.rename('Granulometria'),error], axis=1)


In [10]:
# Recomendación 
def recommendationcc(disp_crusher_index,disp_sag_power_index,disp_ball_work_index,
                     disp_ley_calcopirita,disp_ley_pirita,granulometria,
                     fecha_testeo,
                     fecha_instalacion_revestimientos
                    ):
    '''
    Recibe como input datos de dispatch, granulometría, fecha de consulta y fecha de instalación de revestimientos.
    
    Entrega como output la recomendación de celda de carga.
    '''
        
    # Dato para clustering mineralogia   
    mineralogia=np.array([[disp_crusher_index, disp_sag_power_index, disp_ball_work_index,
                           disp_ley_calcopirita, disp_ley_pirita]])
                         
    
    # Se clasifica mineralogia
    categoria = kmeans.predict(mineralogia)
    
    # Edad SAG
    edad_sag = min((fecha_testeo-fecha_instalacion_revestimientos).days//60,2)

    # Generación del subcontexto
    subcontexto = str(edad_sag) + str(categoria[0])
  
    # Obtención de la recomendación
    consejo=dic[subcontexto].loc[granulometria]["cc"]
    
    return consejo

In [11]:
def generate_data(anio,mes,dia,hr):
    
    '''
    Recibe como input la fecha de consulta, en formato año, mes y dia entero y hora string.
    
    Entrega data necesaria para estructurar la sabana final que permite la visualización de los modelos. 
    Adicionalmente entrega la fecha de consulta.
    '''
    
    # Se recibe año, mes, dia y hora de la consulta
    s1=str(anio)+'-'+str(mes)+'-'+str(dia) # seteo en string de la consulta
    
    h1=hr # Se guarda la hora de consulta
    
    # Fecha consulta
    consulta=parse(s1+' '+h1) 
    
    # Generación de dataframe
    data =df.copy()
    
    # shift de 5 horas data dispatch
    data[tag_dispatch]=data[tag_dispatch].shift(4*5)
    
    # Extracción de tags a usar
    data=data[tag_dispatch+tag_granulometria+tag_metrica].interpolate()
    
    # Rango de visualización
    start_1= consulta-timedelta(hours=5)
    end_1 = consulta+timedelta(hours=3)
    timestamps15_1 = pd.date_range(start_1, end_1, freq='15min')
    
    # Reindex & interpolate
    data=data.reindex(timestamps15_1)
    
    return data,consulta

In [12]:
def generate_data_past(data,consulta,fecha_instalacion_revestimientos):
    
    '''
    Recibe como input data generada generate_data, consulta y ultima fecha de instalación revestimientos.
    
    Entrega como output la data "pasado" para la visualización, esto incluye la recomendación de celda de carga con el tag 
    de granulometría.
    '''
    # Copia de data
    data_forecast=data.copy()
    
    # times 15min 
    start_past= consulta-timedelta(hours=5)
    end_past= consulta
    timestamps15_past = pd.date_range(start_past, end_past, freq='15min')
    
    data_forecast=data.reindex(timestamps15_past)
    
    # Recomendación  con tag granulometria
    data_forecast["cc_tag_gran"]=data_forecast.apply(lambda row: recommendationcc(row["disp_crusher_index"],
                                              row["disp_sag_power_index"],
                                              row["disp_ball_work_index"],
                                              row["disp_ley_calcopirita"],
                                              row["disp_ley_pirita"],
                                              row["CF:CVB007_S2.PNT.FOX"],
                                            row.name,
                                            fecha_instalacion_revestimientos
                                                    ),axis=1)-80
    
    # Definición de status para el posterior concat para generar la sabana que permite la visualización
    data_forecast["status"]="past"
    #data_forecast["consulta"]=consulta
    
    return data_forecast

In [13]:
# recibe como input data del futuro para generar la recomendación a futuro
def generate_recomendation_cc(df,fecha_instalacion_revestimientos):
    
    '''
    Función utilizada para generar las recomendaciones de celda de carga para cada rango de granulometría predecido.
    '''
    
    # Recomendación forecasting esperado
    y_predCC_esperado=df.apply(lambda row: recommendationcc(row["disp_crusher_index"],
                                                  row["disp_sag_power_index"],
                                                  row["disp_ball_work_index"],
                                                  row["disp_ley_calcopirita"],
                                                  row["disp_ley_pirita"],
                                                  row["f80_esperado"],
                                                row.name,
                                                fecha_instalacion_revestimientos
                                                        ),axis=1)-80

    # Recomendación con forecasting limite superior
    y_predCC_sup=df.apply(lambda row: recommendationcc(row["disp_crusher_index"],
                                                  row["disp_sag_power_index"],
                                                  row["disp_ball_work_index"],
                                                  row["disp_ley_calcopirita"],
                                                  row["disp_ley_pirita"],
                                                  row["f80_sup"],
                                                row.name,
                                                fecha_instalacion_revestimientos
                                                        ),axis=1)-80

    # Recomendación con forecasting limite inferior
    y_predCC_inf=df.apply(lambda row: recommendationcc(row["disp_crusher_index"],
                                                  row["disp_sag_power_index"],
                                                  row["disp_ball_work_index"],
                                                  row["disp_ley_calcopirita"],
                                                  row["disp_ley_pirita"],
                                                  row["f80_inf"],
                                                row.name,
                                                fecha_instalacion_revestimientos
                                                        ),axis=1)-80
    
    rec=pd.DataFrame({"cc_esperado":y_predCC_esperado,"cc_sup":y_predCC_sup,"cc_inf":y_predCC_inf})
    
    return rec

In [15]:
def generate_data_future(data,data_past,tag_dispatch,consulta,fecha_instalacion_revestimientos):
    
    '''
    Genera las predicciones de los modelos a partir de data past. Recibe como input tags de dispatch, fecha de consulta y
    fecha de instalación de liners.
    
    Entrega como output todas las predicciones y rangos en un dataframe.
    '''
    # Copia de data
    futuro=data[tag_dispatch+['CF:CVB007_S2.PNT.FOX']].copy()
    futuro.rename(inplace=True,columns={'CF:CVB007_S2.PNT.FOX': "test_F80"})
    
    # times 15min 
    start_future= consulta
    end_future= consulta+timedelta(hours=3)
    timestamps15_future = pd.date_range(start_future, end_future, freq='15min')
    
    # Reindex
    futuro=futuro.reindex(timestamps15_future)
    
    # Definición de status para el posterior concat para generar la sabana que permite la visualización
    futuro["status"]="predict"
    
    # Predict F80 a partir de data past
    ts1=data_past['CF:CVB007_S2.PNT.FOX']
    predict_f80=forecast_granulometry2(ts1, time_forward=3, freq='15T', certeza=65).rename(columns={"Granulometria": 
                                                                                                    "f80_esperado", 
                                                                                        "Lower_Bound": "f80_inf",
                                                                                        "Upper_Bound": "f80_sup"
                                                                                       })
    
    # Join de f80 a la data futuro
    futuro=futuro.join(predict_f80)
    
    # Recomendación CC
    rec_cc=generate_recomendation_cc(futuro,fecha_instalacion_revestimientos)
    
    # join celda de carga
    futuro=futuro.join(rec_cc)
    
    return futuro


In [16]:
def generate_sabana(data_past,data_future,consulta,fecha_instalacion_revestimientos):
    '''
    Genera sabana para su visuaización, contiene fecha de instalación de liners, edad del molino, fecha de consulta,
    data train y test.
    
    '''
    
    sabanita=pd.concat([data_past,data_future],axis=0)
    
    resumen_inst=pd.Series(fecha_instalacion_revestimientos).dt.strftime('%Y-%m-%d')[0]
    edad_sag = min((consulta-fecha_instalacion_revestimientos).days//60,2)
    edad={0:"Nuevo",1:"Medio",2:"Viejo"}
    
    sabanita["inst_liners"]=resumen_inst
    sabanita["consulta"]= consulta
    sabanita["edad_molino"]=edad[edad_sag]
    sabanita.reset_index(drop=False,inplace=True)
    sabanita.rename(inplace=True,columns={"index": "timestamp"})
    return sabanita

In [17]:
def generate_data_dashboard(anio,mes,dia,hr,inicios_campañas):
    '''
    Función que ejecuta todas las funciones auxiliares, entregando los inputs correspondientes según la fecha de consulta.
    
    '''
    
    # Obtención de data preliminar y fecha de consulta en formato timestamp
    data,consulta=generate_data(anio,mes,dia,hr)
    
    # Ultima instalación liners según fecha de consulta
    fecha_instalacion_revestimientos=max(list(filter(lambda x: x<=consulta, inicios_campañas)))
    
    # Se incorpora recomendación de CC con tag de granulometria
    data_past=generate_data_past(data,consulta,fecha_instalacion_revestimientos)
    
    # Se genera forecasting de F80 y recomendación de CC
    data_future=generate_data_future(data,data_past,tag_dispatch,consulta,fecha_instalacion_revestimientos)
    
    # Finalmente se genera data lista para su visualización
    data_dash=generate_sabana(data_past,data_future,consulta,fecha_instalacion_revestimientos)
    
    return data_dash

In [18]:
def generate_visualization(dash):
    '''
    Función que recibe data dash para generar visualización.
    '''
    fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=("TPH y Limite Operario", "Operación Celda de Carga","Granulometría F80","SPI 5 hr"))


    #Valores reales


    fig.add_trace(go.Scatter(x=dash.index, y=dash['CF:215.WIC1605A.MEAS.FOX'], ##FF6511
                        mode='lines',
                        name="TPH real",line=dict(width=3, color="#0BDEEF"),legendgroup = '1'),row=1, col=1)  #1145FF



    fig.add_trace(go.Scatter(x=dash.index, y=dash['CF:EXPERTO:EXP_MS_A.MEAS_5.FOX'],
                        mode='lines',
                        name="Limite operario HH TPH",line=dict(width=3, color="#48EF0B"),legendgroup = '1'),row=1, col=1)



    # Recomendación


    # Inferior
    fig.add_trace(go.Scatter(x=dash.index, y=dash["cc_inf"],
                        mode='lines',
                        name=" ",line=dict(width=0, color="#FF1139"),
                              showlegend=False
                            ),row=2, col=1)

    # Superior
    fig.add_trace(go.Scatter(x=dash.index, y=dash["cc_sup"],
                        mode='lines',
                        name="Rango de Recomendación CC",line=dict(width=0, color="#A6A1AB"),
                             fill='tonexty',legendgroup = '2'),row=2, col=1)



    # Esperado
    fig.add_trace(go.Scatter(x=dash.index, y=dash["cc_esperado"],
                        mode='lines',
                        name="Recomendación CC con forecast F80",
                             line=dict(width=3, color="red"),legendgroup = '2'),row=2, col=1)
    # Sin forecast
    fig.add_trace(go.Scatter(x=dash.index, y=dash["cc_tag_gran"],
                        mode='lines',
                        name="Recomendación CC sin forecast F80",
                             line=dict(width=3, color="#EF0BEC"),legendgroup = '2'),row=2, col=1)


    fig.add_trace(go.Scatter(x=dash.index, y=dash['CF:225.WIC8067.MEAS.FOX'],
                        mode='lines',
                        name="Celda de Carga real",line=dict(width=3, color="#0BDEEF"),legendgroup = '2'),row=2, col=1)  #11FFDC

    fig.add_trace(go.Scatter(x=dash.index, y=dash['CF:EXPERTO:EXP_MS_A.MEAS_1.FOX'],
                        mode='lines',
                        name="Limite operario LL CC",line=dict(width=3, color="#48EF0B"),legendgroup = '2'),row=2, col=1)


    fig.add_trace(go.Scatter(x=dash.index, y=dash['CF:EXPERTO:EXP_MS_A.MEAS_7.FOX'],
                        mode='lines',
                        name="Limite operario HH CC",line=dict(width=3, color="#48EF0B"),legendgroup = '2'),row=2, col=1)


    # Granulometría

    fig.add_trace(go.Scatter(x=dash.index, y=dash["f80_inf"],
                        mode='lines',
                        name=" ",
                        line=dict(width=0, color="red"), showlegend=False
                            ),row=3, col=1)

    fig.add_trace(go.Scatter(x=dash.index, y=dash["f80_sup"],
                             mode='lines',
                        name="Rango de Predicción F80",
                             fill='tonexty',
                             line=dict(width=0, color="#A6A1AB"),legendgroup = '3'
                            ),row=3, col=1
                 )

    fig.add_trace(go.Scatter(x=dash.index, y=dash["f80_esperado"],
                        mode='lines',
                        line=dict(width=3, color="red"),
                        name="Prediction F80",legendgroup = '3'),row=3, col=1)  #D9EF0B

    fig.add_trace(go.Scatter(x=dash.index, y=dash["test_F80"],
                        mode='lines',
                            line=dict(width=3, color="yellow"),
                        name="Test Data F80",legendgroup = '3'),row=3, col=1)


    fig.add_trace(go.Scatter(x=dash.index, y=dash["CF:CVB007_S2.PNT.FOX"],
                        mode='lines',
                        line=dict(width=3, color="#0BDEEF"),
                        name="Train Data F80",legendgroup = '3'),row=3, col=1)


    # SPI
    fig.add_trace(go.Scatter(x=dash.index, y=dash["disp_sag_power_index"],
                        mode='lines',
                        name="SPI",
                             line=dict(width=3, color="red"),legendgroup = '4'),row=4, col=1)



    # Lineas temporales

    fig.add_vrect(x0=dash.index[0], x1=dash["consulta"][0], 
                  annotation_text="Pasado", annotation_position="bottom left",  #
                  fillcolor="#EFE6E6", 
                  opacity=0.25, line_width=0, annotation_font_size=15,)

    fig.add_vrect(x0=dash["consulta"][0], x1=dash.index[-1], 
                  annotation_text="Futuro", annotation_position="bottom right",
                  fillcolor="#0EB9B9",
                  opacity=0.25, line_width=0,annotation_font_size=15)


    # Titulo
    fig.update_layout(title_text="[Fecha consulta: {}] [Última Instalación Liners: {}] [Edad del Molino: {}]".format(
                                                                                                    dash["consulta"][0],
                                                                                                    dash["inst_liners"][0],
                                                                                                        dash["edad_molino"][0]),
                      height=800,
                      width=1000,font=dict(color="white"),template="plotly_dark",legend_tracegroupgap = 90,legend_title="Leyenda")

    # Update xaxis properties
    fig.update_xaxes(tickformat="%H:%M", row=1, col=1)
    fig.update_xaxes(tickformat="%H:%M",row=2, col=1)
    fig.update_xaxes(tickformat="%H:%M",row=3, col=1)
    fig.update_xaxes(title_text="Tiempo - frecuencia 15 Minutos",tickformat="%H:%M",row=4, col=1)

    # Update yaxis properties
    fig.update_yaxes(title_text="TPH", row=1, col=1)
    fig.update_yaxes(title_text="Celda de Carga", row=2, col=1)
    fig.update_yaxes(title_text="% Pasante", row=3, col=1)
    fig.update_yaxes(title_text="SPI", row=4, col=1)
    
    #fig.show()
    return fig

In [19]:
# Ejemplo de Consulta
anio=2020
mes=6
dia=15
hr='13:30'

In [20]:
dash=generate_data_dashboard(anio,mes,dia,hr,inicios_campañas)
dash.set_index("timestamp",inplace=True)

In [21]:
generate_visualization(dash)

In [22]:
def widgets_dashboard(anio,mes,dia,hr):
    # Generación de data para visualización
    dash=generate_data_dashboard(anio,mes,dia,hr,inicios_campañas)
    dash.set_index("timestamp",inplace=True)
    fig=generate_visualization(dash)
    fig.show()
    

In [23]:
# Fechas 15min
date_format = "%Y-%d-%m %H:%M:%S"
start = datetime.strptime("2018-01-01 00:00:00", date_format )
end = datetime.strptime( "2018-02-01 00:00:00", date_format )
timestep = 15
time_freq = str(timestep) + 'min'
timestamps15 = pd.DataFrame(pd.date_range(start, end, freq=time_freq))
timestamps15.rename(columns={0:"Timestamp"},
               inplace=True)
##############

# Dia
options1 = [i for i in range(0,32)]
dia = ipywidgets.Dropdown(
    options=options1,
    value=options1[1],
    description='Día:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Mes
options2 = [i for i in range(1,13)]
mes = ipywidgets.Dropdown(
    options=options2,
    value=options2[1],
    description='Mes:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Año
options3 = ["2019","2020","2021","2022"]
anio = ipywidgets.Dropdown(
    options=options3,
    value=options3[1],
    description='Año:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Hora
horas=pd.Series(timestamps15["Timestamp"]).dt.strftime(' %H:%M ')[0:-1]
options4 = horas
hora = ipywidgets.Dropdown(
    options=options4,
    value=options4[1],
    description='Hora:',
    disabled=False,
    style={'description_width': 'initial'}
)

widget=interactive( widgets_dashboard, dia=dia,mes=mes,anio=anio,hr=hora)
items = [anio,mes,dia,hora]
left_box = VBox([items[0], items[2]])
right_box = VBox([items[1], items[3]])
controls=HBox([left_box, right_box])
output = widget.children[-1]
display(VBox([controls, output]))